# Sentiment Analysis with IMDB movie reviews

In this notebook we will perform sentimend analysis for IMDB movie reviews. This dataset was downloaded from Kaggle, you can download it too in the following link: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews. There are 50k movie revies within the dataset, all of them are labeled. Those labels have values of: negative and positive.

### Importing needed libraries

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

### Reading data & Pre-processing data

As you can see, there are 50k lines and two columns. The sentiment column is our output. This dataset needs some processing because there are special characters and HTML tags within the review, so we will be performing a quick cleaning process before continuing our implementation.

In [3]:
df = pd.read_csv("IMDB Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [7]:
# Separating data into independent and dependent variables
x_data = df['review']
y_data = df['sentiment']

# Cleaning X data
stop_words = set(stopwords.words('english'))    # Getting english stop words to concentrate in the important information
x_data = x_data.replace({'<.*?>': ''}, regex= True) # Removing All the HTML tags
x_data = x_data.replace({'[^A-Za-z]': ''}, regex= True) # Removing everything that doesn't belong to the alphabet
x_data = x_data.apply(lambda review: [w for w in review.split() if w not in stop_words])    # Removing stop words from the reviews
x_data = x_data.apply(lambda review: [w.lower() for w in review])   # Lowercasing all words in the review

# Transformating Y data
y_data = y_data.replace('positive', 1)  # 1 represents positive reviews and 0 represents negative reviews in newer label
y_data = y_data.replace('negative', 0)

# Showing transformed data
print('X Data:\n', x_data)
print('Y Data:\n', y_data)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\teddy/nltk_data'
    - 'd:\\teddy\\Documents\\IMDB-sentiment-analysis-LSTM\\venv\\nltk_data'
    - 'd:\\teddy\\Documents\\IMDB-sentiment-analysis-LSTM\\venv\\share\\nltk_data'
    - 'd:\\teddy\\Documents\\IMDB-sentiment-analysis-LSTM\\venv\\lib\\nltk_data'
    - 'C:\\Users\\teddy\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
